In [1]:
# !pip install tensorflow-datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorflow.keras import layers, models

2025-01-12 20:10:53.867063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 20:10:54.012848: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 20:10:54.819050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-12 20:10:54.819125

In [3]:
#loading mnist data from tensorflow
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
#mnist = tfds.load(name='mnist', split='train', as_supervised=True)
# Returns both train and test split separately
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    image = image
    #image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1] # i believe this is not in the original code
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image)
mnist_test = mnist_test.map(preprocess_image)

2025-01-12 20:11:02.353646: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12688 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:1b:00.0, compute capability: 8.6
2025-01-12 20:11:02.355907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13232 MB memory:  -> device: 1, name: NVIDIA A16, pci bus id: 0000:1c:00.0, compute capability: 8.6
2025-01-12 20:11:02.357967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13232 MB memory:  -> device: 2, name: NVIDIA A16, pci bus id: 0000:1d:00.0, compute capability: 8.6
2025-01-12 20:11:02.360025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13232 MB memory:  -> device: 3, name: NVIDIA A16, pci bus id: 0000:1e:00.0, compute capability: 8.6


In [4]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [5]:
# --------------------------------------------------------
# Hyperparameters & Setup
# --------------------------------------------------------
input_height = 28
input_width = 28
input_channels = 1

# Base Model conv setup
n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

# Dense & Output
n_hidden = 1024
n_out = 10

# --------------------------------------------------------
# 1. Base CNN
# --------------------------------------------------------
def cnn_base(inputs):
    """
    The 'base' CNN model:
      - Conv2D(32, kernel=5) + MaxPool(2)
      - Conv2D(64, kernel=5) + MaxPool(2)
      - Flatten
      - Dense(1024) + Dropout(0.2)
      - Dense(10, softmax)
    """
    conv1 = layers.Conv2D(n_conv1,
                          kernel_size=(conv1_k, conv1_k),
                          strides=stride_conv1,
                          padding="same",
                          activation="relu")(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2,
                          kernel_size=(conv2_k, conv2_k),
                          strides=stride_conv2,
                          padding="same",
                          activation="relu")(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    flatten = layers.Flatten()(pool2)
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # A small dropout for regularization in the base model (can remove if desired)
    dropout = layers.Dropout(0.2)(hidden)

    outputs = layers.Dense(n_out, activation="softmax")(dropout)
    return outputs

# --------------------------------------------------------
# 2. Variation Model (similar complexity)
# --------------------------------------------------------
def cnn_variation(inputs):
    """
    Variation model with similar complexity:
      - 1st Conv has kernel_size=3
      - 2nd Conv has kernel_size=5
      - Dense(1024) + Dropout(0.2)
      - Output(10)
    """
    conv1 = layers.Conv2D(n_conv1,
                          kernel_size=(3, 3),  # changed to 3x3
                          strides=stride_conv1,
                          padding="same",
                          activation="relu")(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2,
                          kernel_size=(conv2_k, conv2_k),  # keep 5x5
                          strides=stride_conv2,
                          padding="same",
                          activation="relu")(pool1)
    pool2 = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    flatten = layers.Flatten()(pool2)
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)
    dropout = layers.Dropout(0.2)(hidden)

    outputs = layers.Dense(n_out, activation="softmax")(dropout)
    return outputs

# --------------------------------------------------------
# 3. More Powerful Model
# --------------------------------------------------------
def cnn_powerful(inputs):
    """
    More powerful model:
      - 3 Conv layers (32, 64, 128) each with kernel=3
      - BatchNormalization after each Conv
      - 2 MaxPools total (or you can add a 3rd if desired)
      - Larger Dense layer (2048) + Dropout(0.3)
      - Output(10)
    """
    # 1st Conv + Pool
    conv1 = layers.Conv2D(32, (3, 3), padding="same", activation="relu")(inputs)
    bn1 = layers.BatchNormalization()(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(bn1)

    # 2nd Conv + Pool
    conv2 = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(pool1)
    bn2 = layers.BatchNormalization()(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(bn2)

    # 3rd Conv (optionally add another pool if you want more downsampling)
    conv3 = layers.Conv2D(128, (3, 3), padding="same", activation="relu")(pool2)
    bn3 = layers.BatchNormalization()(conv3)
    # pool3 = layers.MaxPooling2D(pool_size=(2, 2))(bn3) # uncomment if desired

    flatten = layers.Flatten()(bn3)  # or pool3 if you used a 3rd pooling
    hidden = layers.Dense(2048, activation="relu")(flatten)
    dropout = layers.Dropout(0.3)(hidden)

    outputs = layers.Dense(n_out, activation="softmax")(dropout)
    return outputs

# --------------------------------------------------------
# 4. Simpler Model
# --------------------------------------------------------
def cnn_simple(inputs):
    """
    Simpler model:
      - 1 Conv + Pool (32 filters)
      - Flatten
      - Dense(256)
      - Output(10)
    """
    conv1 = layers.Conv2D(32,
                          kernel_size=(5, 5),
                          strides=1,
                          padding="same",
                          activation="relu")(inputs)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    flatten = layers.Flatten()(pool1)
    hidden = layers.Dense(256, activation="relu")(flatten)
    outputs = layers.Dense(n_out, activation="softmax")(hidden)
    return outputs

# --------------------------------------------------------
# Build Model function
# --------------------------------------------------------
def build_model(model_type="base"):
    """
    Build one of the four models: 'base', 'variation', 'powerful', or 'simple'.
    Returns a tf.keras.Model.
    """
    # Define the input layer
    inputs = tf.keras.Input(shape=(input_height, input_width, input_channels))

    # Build the selected model
    if model_type == "base":
        outputs = cnn_base(inputs)
    elif model_type == "variation":
        outputs = cnn_variation(inputs)
    elif model_type == "powerful":
        outputs = cnn_powerful(inputs)
    elif model_type == "simple":
        outputs = cnn_simple(inputs)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=f"{model_type}_model")
    return model

In [6]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
model=build_model("variation")


In [7]:

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train.batch(100)
test_data = mnist_test.batch(100)
# Get a single batch from the test data (a batch is a tuple of (image, label))
#single_batch = next(iter(test_data))

# single_batch is a tuple, where single_batch[0] contains the image and single_batch[1] contains the label
#image, label = single_batch

# You can print or inspect the first image and label from the batch
#print(image[0].numpy())  # The first image in the batch
#print(label[0].numpy())  # The label for the first image


In [8]:
model.fit(train_data, epochs=15)

Epoch 1/15


2025-01-12 20:11:05.118957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2025-01-12 20:11:06.582127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


600/600 [==============================] - 12s 11ms/step - loss: 0.8433 - accuracy: 0.9295
Epoch 2/15
600/600 [==============================] - 6s 11ms/step - loss: 0.0639 - accuracy: 0.9799
Epoch 3/15
600/600 [==============================] - 7s 11ms/step - loss: 0.0443 - accuracy: 0.9858
Epoch 4/15
600/600 [==============================] - 7s 11ms/step - loss: 0.0366 - accuracy: 0.9883
Epoch 5/15
600/600 [==============================] - 6s 10ms/step - loss: 0.0292 - accuracy: 0.9905
Epoch 6/15
600/600 [==============================] - 6s 10ms/step - loss: 0.0315 - accuracy: 0.9899
Epoch 7/15
600/600 [==============================] - 6s 11ms/step - loss: 0.0250 - accuracy: 0.9920
Epoch 8/15
600/600 [==============================] - 6s 11ms/step - loss: 0.0266 - accuracy: 0.9919
Epoch 9/15
600/600 [==============================] - 6s 11ms/step - loss: 0.0201 - accuracy: 0.9939
Epoch 10/15
600/600 [==============================] - 6s 11ms/step - loss: 0.0219 - accuracy: 0.9936

In [9]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

100/100 [==============================] - 2s 12ms/step - loss: 0.0782 - accuracy: 0.9864
Test accuracy: 0.9864000082015991
summary: 
Model: "variation_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                             

In [10]:
model.save('CNNModel/variationCNN.keras')